# Alpha Test

Test ideas relating to alpha racing

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os

import numpy as np

from math import pi, cos, sqrt

from common_core import Printable, projdir, loadCsv

## Proximity Testing

Functions to calculate the proximity between two points.

See the 'haversine_vs_pythagoras' notebook for a full comparison of the Haversine Formula vs Pythagorean Theorem.

The code here is very similar but makes use of pre-calculated cache values.

In [2]:
EARTH_RADIUS = 6371009


def estimateProximity(latitude1r, longitude1r, latitude2r, longitude2r, radius=EARTH_RADIUS):
    '''Estimate the Euclidean distance between two points on a sphere using Pythagorean Theorem'''

    # Calculate distance north / south. n.b. Can cache the values of y1 + y2
    y1 = latitude1r * EARTH_RADIUS
    y2 = latitude2r * EARTH_RADIUS
    yDelta = y2 - y1

    # For the sake of completeness we need to cope with the points either side of the 180th meridian
    longDelta = abs(longitude2r - longitude1r)

    # Note: This adjustment has not been factored into the "cost" comparison above because it is so unlikely to occur!
    if longDelta > pi:
        longDelta -= 2 * pi

    # Calculate distance east / west. n.b. Can cache EARTH_RADIUS * cos(latitude2r)
    xDelta = longDelta * EARTH_RADIUS * cos(latitude2r)

    # Apply Pythagorean theorem to determine the distance
    distance = sqrt(xDelta ** 2 + yDelta ** 2)

    return distance

## Process Track

In [3]:
def calculateExtras(track):
    '''Calculate all of the extra fields for the track'''

    extras = {}

    # Convert timestamps into datetimes
    extras['datetime'] = np.datetime64('1989-12-31T00:00') + track['timestamp'].astype('timedelta64[s]')
    
    # Convert semicircles to radians
    extras['latitude'] = np.radians(track['position_lat'] * (180.0 / 2 ** 31))
    extras['longitude'] = np.radians(track['position_long'] * (180.0 / 2 ** 31))

    # Calculate distance above/below equator
    extras['yOffset'] = extras['latitude'] * EARTH_RADIUS
    
    # Calculate cumulative distance (m) using speed (m/s)
    extras['cumulativeDistance'] = track['speed'].cumsum()
    
    return extras


    
def processTrack(filename):
    '''Process the track'''

    # Load the track and calculate the extra fields
    track = loadCsv(filename)   
    extras = calculateExtras(track)

    # Find the start and the end of the fastest alpha
    datetimes = extras['datetime']
    idx1 = np.where(datetimes == np.datetime64('2022-04-04T12:55:25'))[0][0]
    idx2 = np.where(datetimes == np.datetime64('2022-04-04T12:56:08'))[0][0]
    
    # Calculate the proximity between the two points
    latitudes = extras['latitude']
    longitudes = extras['longitude']
    proximity = estimateProximity(latitudes[idx1], longitudes[idx1], latitudes[idx2], longitudes[idx2])
    
    print('Proximity = {:0.3f} metres'.format(proximity))

## Run Tests

In [4]:
if __name__ == '__main__':
    processTrack(os.path.join(projdir, 'sessions', '20220404', '20220404-apex-pro.csv'))

Proximity = 46.496 metres
